<a href="https://colab.research.google.com/github/starminalush/mlops_report/blob/main/tensorrt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorRT

! Работает на тензорных ядрах, поэтому нужна тесла

Поэтому сначала поставим tensorrt, лучше из [архива](https://developer.nvidia.com/compute/machine-learning/tensorrt/secure/7.2.2/tars/TensorRT-7.2.2.3.Ubuntu-18.04.x86_64-gnu.cuda-11.1.cudnn8.0.tar.gz). Скачайте его и поместите на drive, затем выполните следующие команды

In [2]:
!nvidia-smi

Sun Apr 24 08:50:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!uname -m

x86_64


In [4]:
!tar -xzvf /content/drive/MyDrive/TensorRT-8.2.1.8.Linux.x86_64-gnu.cuda-11.4.cudnn8.2.tar.gz

TensorRT-8.2.1.8/
TensorRT-8.2.1.8/TensorRT-Release-Notes.pdf
TensorRT-8.2.1.8/onnx_graphsurgeon/
TensorRT-8.2.1.8/onnx_graphsurgeon/onnx_graphsurgeon-0.3.12-py2.py3-none-any.whl
TensorRT-8.2.1.8/samples/
TensorRT-8.2.1.8/samples/sampleNMT/
TensorRT-8.2.1.8/samples/sampleNMT/model/
TensorRT-8.2.1.8/samples/sampleNMT/model/likelihoodCombinationOperator.h
TensorRT-8.2.1.8/samples/sampleNMT/model/contextNMT.cpp
TensorRT-8.2.1.8/samples/sampleNMT/model/multiplicativeAlignment.h
TensorRT-8.2.1.8/samples/sampleNMT/model/slpAttention.h
TensorRT-8.2.1.8/samples/sampleNMT/model/decoder.h
TensorRT-8.2.1.8/samples/sampleNMT/model/lstmEncoder.cpp
TensorRT-8.2.1.8/samples/sampleNMT/model/beamSearchPolicy.h
TensorRT-8.2.1.8/samples/sampleNMT/model/beamSearchPolicy.cpp
TensorRT-8.2.1.8/samples/sampleNMT/model/componentWeights.h
TensorRT-8.2.1.8/samples/sampleNMT/model/embedder.h
TensorRT-8.2.1.8/samples/sampleNMT/model/encoder.h
TensorRT-8.2.1.8/samples/sampleNMT/model/softmaxLikelihood.h
TensorRT-8.

In [5]:
!ls TensorRT-8.2.1.8

bin   graphsurgeon  onnx_graphsurgeon  targets
data  include	    python	       TensorRT-Release-Notes.pdf
doc   lib	    samples	       uff


In [6]:
!python --version

Python 3.7.13


In [7]:
!python3 -m pip install TensorRT-8.2.1.8/python/tensorrt-8.2.1.8-cp37-none-linux_x86_64.whl

Processing ./TensorRT-8.2.1.8/python/tensorrt-8.2.1.8-cp37-none-linux_x86_64.whl


In [8]:
!pip install pycuda pytools==2022.1.1

     |████████████████████████████████| 1.7 MB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 67 kB 5.7 MB/s 
     |████████████████████████████████| 78 kB 6.3 MB/s 
  Created wheel for pytools: filename=pytools-2022.1.1-py2.py3-none-any.whl size=63859 sha256=754efa4f05542ebb31cbdfce1d0e724644923640d16d62da1d382a3bc19c0417
  Stored in directory: /root/.cache/pip/wheels/ee/a2/73/6f43d4f99ec69f7731617d44b68bf85aa66fc2da7b33cbe16b
  Created wheel for pycuda: filename=pycuda-2021.1-cp37-cp37m-linux_x86_64.whl size=626634 sha256=ccf320f696a67cccfd3f79589bd2db028eb518f167ecc9b915be8afd59f059bb
  Stored in directory: /root/.cache/pip/wheels/c4/ef/49/dc6a5feb8d980b37c83d465ecab24949a6aa19458522a9e001
Successfully built pytools pycuda


In [9]:
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:TensorRT-8.2.1.8/lib

А теперь будем экспортировать модель в tensorrt

In [10]:
!sudo apt-get install libnvinfer8 libnvonnxparsers8 libnvparsers8 libnvinfer-dev libnvinfer-plugin8

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  libnvinfer-dev libnvinfer-plugin8 libnvinfer8 libnvonnxparsers8
  libnvparsers8
0 upgraded, 5 newly installed, 0 to remove and 40 not upgraded.
Need to get 374 MB of archives.
After this operation, 1,293 MB of additional disk space will be used.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  libnvinfer8 8.2.4-1+cuda11.4 [167 MB]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  libnvinfer-dev 8.2.4-1+cuda11.4 [194 MB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  libnvinfer-plugin8 8.2.4-1+cuda11.4 [12.4 MB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  libnvonnxparsers8 8.2.4-1+cuda11.4 [700 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  libnvparsers8 8.2.4-1+cuda11.4 

In [11]:
!pip install onnx transformers onnxruntime-gpu folium==0.2.1

     |████████████████████████████████| 12.8 MB 5.7 MB/s 
     |████████████████████████████████| 4.0 MB 33.8 MB/s 
     |████████████████████████████████| 108.9 MB 6.5 kB/s 
     |████████████████████████████████| 69 kB 7.0 MB/s 
     |████████████████████████████████| 77 kB 6.1 MB/s 
     |████████████████████████████████| 596 kB 43.5 MB/s 
     |████████████████████████████████| 6.6 MB 35.0 MB/s 
     |████████████████████████████████| 895 kB 44.8 MB/s 
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79808 sha256=3963e72f9c561e2a8fe9c9e093ab886c9d5f47b92cbd245fd7af4eb6406f1312
  Stored in directory: /root/.cache/pip/wheels/9a/f0/3a/3f79a6914ff5affaf50cabad60c9f4d565283283c97f0bdccf
Successfully built folium
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: folium
    Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:

In [12]:
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np
import tensorrt as trt

In [13]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast
from transformers.onnx import export
from pathlib import Path
from typing import Mapping, OrderedDict
from transformers.onnx import OnnxConfig
from transformers import AutoConfig
import onnxruntime as nxrun
import onnx
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd
from torch.nn.utils import prune

In [14]:
tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)

@torch.no_grad()
def predict(text):
    inputs = tokenizer(text, max_length=16, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted[0]

Downloading:   0%|          | 0.00/499 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/943 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/679M [00:00<?, ?B/s]

конвертируем модель в onnx под cuda

In [15]:
text = 'Рандомный текст'

In [16]:

from torch.onnx import TrainingMode

dummy_input0 = torch.randint(1, 500, (1,512)).to('cuda')
dummy_input1 = torch.randint(0, 1, (1,512 )).to('cuda')
dummy_input2 =  torch.randint(0, 1, (1,512 )).to('cuda')
dummy_inputs = (dummy_input0,dummy_input1,dummy_input2)
with torch.no_grad():
  symbolic_names = {0:'batch_size', 1: 'max_seq_len'} 
  torch.onnx.export(model.to('cuda'),               # model being run
                    dummy_inputs,                         # model input (or a tuple for multiple inputs)
                    "model.onnx",   # where to save the model (can be a file or file-like object)
                    # export_params=True,        # store the trained parameter weights inside the model file
                    opset_version=11,          # the ONNX version to export the model to
                    do_constant_folding=True,
                    input_names = ["input_ids","attention_mask","token_type_ids"],
                    output_names = ['output'],# whether to execute constant folding for optimization
                    dynamic_axes={'input_ids': symbolic_names,        # variable length axes
                                  'attention_mask' : symbolic_names,
                                  'token_type_ids' : symbolic_names},
                    training=TrainingMode.EVAL
                    )

In [17]:
sess_options = nxrun.SessionOptions()
providers = [
    'CUDAExecutionProvider'
]

model_ONNX = nxrun.InferenceSession("model.onnx", sess_options, providers)

In [18]:
def predict_onnx(text):
  inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='np')
  outputs  = model_ONNX.run(None, dict(inputs))[0][0]
  result = np.where(outputs == np.amax(outputs))[0][0]
  return result

In [19]:
text = 'Как задолбали эти тупые правила: не есть кота, не бить посуду, не есть кота'

In [20]:
predict_onnx(text)

2

In [21]:
import onnx
model = onnx.load("model.onnx")
result = onnx.checker.check_model(model)
result

https://lifesaver.codes/answer/tensorrt-error-network-must-have-at-least-one-output-183

In [28]:
from typing import Callable, Dict, List, OrderedDict, Tuple

import tensorrt as trt
import torch
from tensorrt import ICudaEngine, IExecutionContext
from tensorrt.tensorrt import (
    Builder,
    IBuilderConfig,
    IElementWiseLayer,
    ILayer,
    INetworkDefinition,
    IOptimizationProfile,
    IReduceLayer,
    Logger,
    OnnxParser,
    Runtime,
)

def build_engine(
    runtime: Runtime,
    onnx_file_path: str,
    logger: Logger,
    min_shape: Tuple[int, int],
    optimal_shape: Tuple[int, int],
    max_shape: Tuple[int, int],
    workspace_size: int,
#     fp16: bool,
#     int8: bool,
) -> ICudaEngine:
    """
    Convert ONNX file to TensorRT engine.
    It supports dynamic shape, however it's advised to keep sequence length fix as it hurts performance otherwise.
    Dynamic batch size don't hurt performance and is highly advised.
    :param runtime: global variable shared accross inference call / model building
    :param onnx_file_path: path to the ONNX file
    :param logger: specific logger to TensorRT
    :param min_shape: the minimal shape of input tensors. It's advised to set first dimension (batch size) to 1
    :param optimal_shape: input tensor shape used for optimizations
    :param max_shape: maximal input tensor shape
    :param workspace_size: GPU memory to use during the building, more is always better. If there is not enough memory,
    some optimization may fail, and the whole conversion process will crash.
    :param fp16: enable FP16 precision, it usually provide a 20-30% boost compared to ONNX Runtime.
    :param int8: enable INT-8 quantization, best performance but model should have been quantized.
    :return: TensorRT engine to use during inference
    """
    with trt.Builder(logger) as builder:  # type: Builder
        with builder.create_network(
            flags=1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)
        ) as network_definition:  # type: INetworkDefinition
            with trt.OnnxParser(network_definition, logger) as parser:  # type: OnnxParser
                builder.max_batch_size = max_shape[0]  # max batch size
                config: IBuilderConfig = builder.create_builder_config()
                config.max_workspace_size = workspace_size
                # to enable complete trt inspector debugging, only for TensorRT >= 8.2
                # config.profiling_verbosity = trt.ProfilingVerbosity.DETAILED
                # disable CUDNN optimizations
                config.set_tactic_sources(
                    tactic_sources=1 << int(trt.TacticSource.CUBLAS) | 1 << int(trt.TacticSource.CUBLAS_LT)
                )
#                 if int8:
#                     config.set_flag(trt.BuilderFlag.INT8)
#                 if fp16:
#                     config.set_flag(trt.BuilderFlag.FP16)
                config.set_flag(trt.BuilderFlag.DISABLE_TIMING_CACHE)
                # https://github.com/NVIDIA/TensorRT/issues/1196 (sometimes big diff in output when using FP16)
                config.set_flag(trt.BuilderFlag.OBEY_PRECISION_CONSTRAINTS)
                with open(onnx_file_path, "rb") as f:
                    parsed = parser.parse(f.read())
                print(parsed)
                profile: IOptimizationProfile = builder.create_optimization_profile()
                for num_input in range(network_definition.num_inputs):
                    profile.set_shape(
                        input=network_definition.get_input(num_input).name,
                        min=min_shape,
                        opt=optimal_shape,
                        max=max_shape,
                    )
                config.add_optimization_profile(profile)
#                 if fp16:
#                     network_definition = fix_fp16_network(network_definition)
                trt_engine = builder.build_serialized_network(network_definition, config)
                engine: ICudaEngine = runtime.deserialize_cuda_engine(trt_engine)
                assert engine is not None, "error during engine generation, check error messages above :-("
                with open(trt_model_name, "wb") as f:
                  f.write(engine.serialize())
                return engine

In [33]:
from pathlib import Path

trt_logger: Logger = trt.Logger(trt.Logger.ERROR)
runtime: Runtime = trt.Runtime(trt_logger)
trt_model_name = "model.plan"

# create only of does not exist because it's slow to run...
if not Path(trt_model_name).exists():
    engine: ICudaEngine = build_engine(
        runtime=runtime,
        onnx_file_path="model.onnx",
        logger=trt_logger,
        min_shape=(1, 1),
        optimal_shape=(1, 128),  # num beam, batch size
        max_shape=(1, 512),  # num beam, batch size
        workspace_size=1024 * 1024**2,
#         fp16=True,
#         int8=False,
    )
#     save_engine(engine, trt_model_name)
    

True


In [26]:
engine

Начинаются страдания и боль

1. Загружаем модель

In [30]:
text = 'Как задолбали эти тупые правила: не есть кота, не бить посуду, не есть кота'

In [34]:
import pycuda.driver as cuda
import pycuda.autoinit
import collections
import numpy as np
import time
from datetime import datetime

TRT_LOGGER = trt.Logger()
inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
with open("model.plan", "rb") as f, \
    trt.Runtime(TRT_LOGGER) as runtime, \
    runtime.deserialize_cuda_engine(f.read()) as engine, \
    engine.create_execution_context() as context:

     # We always use batch size 1.
    input_shape = (1, 512)
    input_nbytes = trt.volume(input_shape) * trt.int32.itemsize
    
    # Allocate device memory for inputs.
    d_inputs = [cuda.mem_alloc(input_nbytes) for binding in range(3)]
    # Create a stream in which to copy inputs/outputs and run inference.
    stream = cuda.Stream()

    # Specify input shapes. These must be within the min/max bounds of the active profile (0th profile in this case)
    # Note that input shapes can be specified on a per-inference basis, but in this case, we only have a single shape.
    for binding in range(3):
        context.set_binding_shape(binding, input_shape)
    assert context.all_binding_shapes_specified

    # Allocate output buffer by querying the size from the context. This may be different for different input shapes.
    h_output = cuda.pagelocked_empty(tuple(context.get_binding_shape(3)), dtype=np.float32)
    d_output = cuda.mem_alloc(h_output.nbytes)

    print("\nRunning Inference...")
    start = datetime.now()

    _NetworkOutput = collections.namedtuple(  # pylint: disable=invalid-name
        "NetworkOutput",
        ["output"])
    networkOutputs = []

    # Copy inputs
    input_ids = cuda.register_host_memory(np.ascontiguousarray(inputs.input_ids))
    input_mask = cuda.register_host_memory(np.ascontiguousarray(inputs.attention_mask))
    segment_ids = cuda.register_host_memory(np.ascontiguousarray(inputs.token_type_ids))

    cuda.memcpy_htod_async(d_inputs[0], input_ids, stream)
    cuda.memcpy_htod_async(d_inputs[2], segment_ids, stream)
    cuda.memcpy_htod_async(d_inputs[1], input_mask, stream)

    # Run inference
    context.execute_async_v2(bindings=[int(d_inp) for d_inp in d_inputs] + [int(d_output)], stream_handle=stream.handle)
    # Synchronize the stream
    stream.synchronize()
    # Transfer predictions back from GPU
    cuda.memcpy_dtoh_async(h_output, d_output, stream)
    stream.synchronize()
    h_output = h_output[0]
    result = np.where(h_output == np.amax(h_output))[0][0]
    print(result)
    print(datetime.now() - start)


Running Inference...
2
0:00:00.103901
